# Section 3
## Read in database credentials
Below the credentials for connecting to the database are read into variables by extracting the lines from the local file. The local file is not included in the repo.

In [1]:
db_name = ""
db_user = ""
db_pass = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_name = f.readline().strip()
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_host = f.readline().strip()

## Shorthand connect to database
This method will just return a new database connection with the default credentials made available above.

In [2]:
import pymysql as pms

In [3]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Test connection
The next code segment tests to ensure that the database connection is working properly.

In [4]:
try:
    con = get_connect()
    print("Successfully connected")
finally:
    if con:
        print("Closing connection")
        con.close()

Successfully connected
Closing connection


In [5]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Shorthand query execution and output
The method below accepts a single parameter (expected query), executes the parameter as a SQL query, and outputs the results. The connection is closed before the function terminates.

In [6]:
def execute_sql_output_result(query_string):
    """
    Given the query_string parameter, this function connects to the database, executes
    the query, outputs the result, and closes the connection.
    """
    try:
        con = get_connect()
        with con.cursor() as cur:
            #If the query_string is a single string, execute the string
            if type(query_string) == str:
                cur.execute(query_string)
                result = cur.fetchall()
                print("=== {} RESULTS ===".format(len(result)))
                #Column names
                print(" ".join([i[0] for i in cur.description]))
                #Results
                for i in range(len(result)):
                    print("{}: {}".format(i, result[i]))
    finally:
        if con:
            con.close()

## Single Row Functions (SRFs)
Single row functions can be used to perform functions on multiple entries 1 row at a time.

### CONCAT

In [7]:
execute_sql_output_result("""
    SELECT CONCAT('$', sal) AS Salary FROM emp;
""")

=== 14 RESULTS ===
Salary
0: ('$800.00',)
1: ('$1600.00',)
2: ('$1250.00',)
3: ('$2975.00',)
4: ('$1250.00',)
5: ('$2850.00',)
6: ('$2450.00',)
7: ('$3000.00',)
8: ('$5000.00',)
9: ('$1500.00',)
10: ('$1100.00',)
11: ('$950.00',)
12: ('$3000.00',)
13: ('$1300.00',)


### UPPER

In [8]:
execute_sql_output_result("""
    SELECT UPPER('hello') FROM emp;
""")

=== 14 RESULTS ===
UPPER('hello')
0: ('HELLO',)
1: ('HELLO',)
2: ('HELLO',)
3: ('HELLO',)
4: ('HELLO',)
5: ('HELLO',)
6: ('HELLO',)
7: ('HELLO',)
8: ('HELLO',)
9: ('HELLO',)
10: ('HELLO',)
11: ('HELLO',)
12: ('HELLO',)
13: ('HELLO',)


### LOWER

In [9]:
execute_sql_output_result("""
    SELECT LOWER(job) AS "Lowercase Job" FROM emp
""")

=== 14 RESULTS ===
Lowercase Job
0: ('clerk',)
1: ('salesman',)
2: ('salesman',)
3: ('manager',)
4: ('salesman',)
5: ('manager',)
6: ('manager',)
7: ('analyst',)
8: ('president',)
9: ('salesman',)
10: ('clerk',)
11: ('clerk',)
12: ('analyst',)
13: ('clerk',)


## DUAL table
The DUAL table can be used to test out SRFs.

In [10]:
execute_sql_output_result("""
    SELECT UPPER('dog') FROM dual;
""")

=== 1 RESULTS ===
UPPER('dog')
0: ('DOG',)


## More SRFs
Here are some more single row functions.

### INITCAP
`INITCAP` capitalizes the first character in each word.

In [11]:
sql_syntax = """
    SELECT INITCAP('dogs go to heaven if they are good dogs') FROM dual;
"""
#INITCAP is not available in MySQL
#execute_sql_output_result(sql_syntax)

### LENGTH
`LENGTH` returns the length of a string type value.

In [12]:
some_string = "tigers go roar meow"
sql_syntax = """
    SELECT LENGTH('{}') AS "Length of '{}'" FROM dual;
""".format(some_string, some_string)
execute_sql_output_result(sql_syntax)

=== 1 RESULTS ===
Length of 'tigers go roar meow'
0: (19,)


### SUBSTR
`SUBSTR` extracts a substring from a list of characters. This string is indexed starting from 1, not 0.

In [13]:
start_index = 6
chars_to_extract = 50
#Note: SUBSTRING is also acceptable: https://www.w3schools.com/sql/func_mysql_substring.asp
sql_syntax = """
    SELECT SUBSTR('{}', {}, {}) AS "Substring" FROM dual;
""".format(some_string, start_index, chars_to_extract)
execute_sql_output_result(sql_syntax)

=== 1 RESULTS ===
Substring
0: ('s go roar meow',)


### LPAD and RPAD
`LPAD` and `RPAD` can be used to pad a string left or right, to a specified length, using a specific padding character.

In [14]:
length = 10
origin_string = "hi"
pad_char = '$'
execute_sql_output_result("""
    SELECT LPAD('{}', {}, '{}'), RPAD('{}', {}, '{}') FROM dual;
""".format(origin_string, length, pad_char, origin_string, length, pad_char))

=== 1 RESULTS ===
LPAD('hi', 10, '$') RPAD('hi', 10, '$')
0: ('$$$$$$$$hi', 'hi$$$$$$$$')


### LTRIM and RTRIM
These methods can be used to trim data off of an character string.

In [15]:
origin_string = "aababaaa"
trim_char = "a"
sql_syntax = """
    SELECT LTRIM('{}', '{}'), RTRIM('{}', '{}') AS Trimmed FROM dual;
""".format(origin_string, trim_char, origin_string, trim_char)
#LTRIM in MySQL removes leading white space
#execute_sql_output_result(sql_syntax)
execute_sql_output_result("""
    SELECT TRIM(LEADING '{}' FROM '{}'), TRIM(TRAILING '{}' FROM '{}'), TRIM(BOTH '{}' FROM '{}')
        FROM dual;
""".format(trim_char, origin_string, trim_char, origin_string, trim_char, origin_string))

=== 1 RESULTS ===
TRIM(LEADING 'a' FROM 'aababaaa') TRIM(TRAILING 'a' FROM 'aababaaa') TRIM(BOTH 'a' FROM 'aababaaa')
0: ('babaaa', 'aabab', 'bab')


## NUMERIC SRFs
### ROUND
The round function rounds a numeric to a specified number of decimals.

In [16]:
execute_sql_output_result("""
    SELECT ROUND(123.456, 2) FROM dual;
""")
print()
execute_sql_output_result("""
    SELECT ROUND(123.456, 8) FROM dual;
""")
print()
execute_sql_output_result("""
    SELECT ROUND(123.456) FROM dual;
""")

=== 1 RESULTS ===
ROUND(123.456, 2)
0: (Decimal('123.46'),)

=== 1 RESULTS ===
ROUND(123.456, 8)
0: (Decimal('123.45600000'),)

=== 1 RESULTS ===
ROUND(123.456)
0: (Decimal('123'),)


### TRUNC
`TRUNC` provides numeric truncation to a specified number of decimal places (not rounding, just digit removal).

In [17]:
#Causes error in MySQL
sql_syntax = """
    SELECT TRUNC(123.455273) FROM dual;
"""

execute_sql_output_result("""
    SELECT TRUNCATE(123.655273, 0) FROM dual;
""")
print()
execute_sql_output_result("""
    SELECT ROUND(123.655273, 0) FROM dual;
""")

=== 1 RESULTS ===
TRUNCATE(123.655273, 0)
0: (Decimal('123'),)

=== 1 RESULTS ===
ROUND(123.655273, 0)
0: (Decimal('124'),)


## DATE SRFs
### SYSDATE
`SYSDATE` returns the current date for the installed OS: https://www.w3schools.com/sql/func_mysql_sysdate.asp.

In [18]:
execute_sql_output_result("""
    SELECT CONCAT('', SYSDATE()) FROM dual;
""")

=== 1 RESULTS ===
CONCAT('', SYSDATE())
0: ('2018-03-24 05:09:25',)


### ADD_MONTHS (DATE_ADD) and DATE_FORMAT
`ADD_MONTHS` allows you to add a specified number of months to a date value. `DATE_FORMAT` allows you to format a date value to a specific style.

In [19]:
execute_sql_output_result("""
    SELECT DATE_FORMAT(DATE_ADD(CONCAT('', NOW()), INTERVAL -36 MONTH), '%m/%d/%Y') FROM dual;
""")

=== 1 RESULTS ===
DATE_FORMAT(DATE_ADD(CONCAT('', NOW()), INTERVAL -36 MONTH), '%m/%d/%Y')
0: ('03/24/2015',)


## Even more SRFs
### TO_CHAR (DATE_FORMAT)
The `TO_CHAR` function can convert a date or number to a string. The equivalent in MySQL is `DATE_FORMAT`.

In [41]:
execute_sql_output_result("""
    SELECT DATE_FORMAT(NOW(), "%M %D, %Y") FROM dual;
""")
print()
execute_sql_output_result("""
    SELECT DATE_FORMAT(NOW(), "%m %d, %Y") FROM dual;
""")

=== 1 RESULTS ===
DATE_FORMAT(NOW(), "%M %D, %Y")
0: ('March 24th, 2018',)

=== 1 RESULTS ===
DATE_FORMAT(NOW(), "%m %d, %Y")
0: ('03 24, 2018',)


### TO_CHAR for numbers

In [50]:
execute_sql_output_result("""
    SELECT ename AS "Employee Name", CONCAT('$', FORMAT(sal, 0)) "Formatted Salary", CONCAT('$', sal) AS Salary FROM emp LIMIT 3;
""")

=== 3 RESULTS ===
Employee Name Formatted Salary Salary
0: ('SMITH', '$800', '$800.00')
1: ('ALLEN', '$1,600', '$1600.00')
2: ('WARD', '$1,250', '$1250.00')


## NVL and IFNULL
The `NVL` function can be used to replace null-valued columns with filler values. The `IFNULL` function is equivalent in MySQL.

In [55]:
execute_sql_output_result("""
    SELECT empno, ename, sal, IFNULL(comm, 'No data found.') AS commIFNULL, comm FROM emp WHERE comm IS NULL
""")

=== 10 RESULTS ===
empno ename sal commIFNULL comm
0: (7369, 'SMITH', 800.0, 'No data found.', None)
1: (7566, 'JONES', 2975.0, 'No data found.', None)
2: (7698, 'BLAKE', 2850.0, 'No data found.', None)
3: (7782, 'CLARK', 2450.0, 'No data found.', None)
4: (7788, 'SCOTT', 3000.0, 'No data found.', None)
5: (7839, 'KING', 5000.0, 'No data found.', None)
6: (7876, 'ADAMS', 1100.0, 'No data found.', None)
7: (7900, 'JAMES', 950.0, 'No data found.', None)
8: (7902, 'FORD', 3000.0, 'No data found.', None)
9: (7934, 'MILLER', 1300.0, 'No data found.', None)
